In [61]:
from dotenv import load_dotenv
import os
import json
import pandas as pd
import base64
from requests import post, get
import csv

load_dotenv()

client_id = os.getenv("CID")
client_secret = os.getenv("SECRET")

In [62]:
def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }

    data = {"grant_type": "client_credentials"}

    result = post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]

    return token

def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

In [63]:
import random

def getRandomSearch():
    # Una lista de todos los caracteres que se pueden elegir.
    characters = 'abcdefghijklmnopqrstuvwxyz'
    
    # Obtiene un carácter aleatorio de la cadena de caracteres.
    randomCharacter = random.choice(characters)
    randomSearch = ''
    
    # Coloca el carácter comodín al principio, o al principio y al final, de forma aleatoria.
    if random.randint(0, 1) == 0:
        randomSearch = randomCharacter + '%'
    else:
        randomSearch = '%' + randomCharacter + '%'
    
    return randomSearch

def search_banch_of_tracks(randomSearch, randomOffset, limit=50):
    token = get_token()
    auth_header = get_auth_header(token)
    url = "https://api.spotify.com/v1/search"
    params = {
        "q": randomSearch,
        "type": "track",
        "limit": limit,
        "offset": randomOffset,
    }
    response = get(url, headers=auth_header, params=params)
    json_response = json.loads(response.content)
    return json_response

def write_csv(json_response):
    try:
        new_tracks_df = pd.DataFrame(json_response["tracks"]["items"])
        with open("db_extract.csv", "a") as f:
            new_tracks_df.to_csv(f)
    except:
        pass
        
def depurar_csv():
    df = pd.read_csv("db_extract.csv")
    df.drop_duplicates(subset=["id"], inplace=True)
    df.to_csv("db_extract.csv", index=False)

In [64]:
def get_several_tracks(iterations=1000):
    for i in range(0, iterations):
        randomSearch = getRandomSearch()
        randomOffset = random.randint(0, 1000)
        print(f'Iteration {i} - Search: {randomSearch} - Offset: {randomOffset}')
        json_response = search_banch_of_tracks(randomSearch, randomOffset)
        write_csv(json_response)
    depurar_csv()

In [65]:
get_several_tracks(iterations=10000)

Iteration 0 - Search: e% - Offset: 293
Iteration 1 - Search: l% - Offset: 784
Iteration 2 - Search: %w% - Offset: 584


KeyboardInterrupt: 

In [60]:
df = pd.read_csv("db_extract.csv")
df.keys()

Index(['Unnamed: 0', 'album', 'artists', 'available_markets', 'disc_number',
       'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href',
       'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number',
       'type', 'uri'],
      dtype='object')